In [ ]:
import innvestigate
import numpy as np
from nilearn.image import load_img,mean_img
from keras.models import load_model
import nibabel as nib
import create_dataset,utils
import pandas as pd
from nilearn.image import resample_img,concat_imgs,load_img,resample_to_img


In [ ]:
data_path = '/home/ubuntu/hacking/data/stephan-data-ni-files'
model_to_use = '/home/ubuntu/hacking/projects/deep-mediation/jan-2021/results-feb/model-iter-1.h5'
img_to_use = '/home/ubuntu/hacking/data/stephan-data-ni-files/BMRK5_data/stim_bmrk5_S860_OC1339_zs.nii'

In [ ]:
dataset = create_dataset.generate_dataset(data_path,test_data_size=0.30)
test_rate,test_temp,test_imgs_list,flat_test_rate,flat_test_rate_zs = utils.get_rate_temp_img(dataset,subjs='test_subjs')
df_test = pd.DataFrame()
df_test['X'] = test_temp
df_test['Y'] = flat_test_rate_zs

In [ ]:
print("Reading testing images...")
test_imgs = concat_imgs(test_imgs_list)
test_imgs = np.rollaxis(test_imgs.get_fdata(), 3, 0)[...,None]
print(test_imgs.shape)

In [ ]:
def get_model(model_to_use):
    model_with_gpu = load_model(model_to_use)
    model = model_with_gpu.layers[-2]
    return model
def get_nifti_params(input_img):
    affine = input_img.affine
    hdr = input_img.header
    return affine,hdr
def get_nifti_image(array,affine,header):
    nii_image = nib.Nifti1Image(array, affine, header)
    return nii_image

In [ ]:
# input_img = load_img(img_to_use)
# affine,hdr = get_nifti_params(input_img)
# input_img = input_img.get_fdata()
# input_img = np.rollaxis(input_img, 3,0)[...,None]
# print(input_img.shape)

In [ ]:
model=get_model(model_to_use)
model.summary()

In [ ]:
analyzer = innvestigate.create_analyzer("gradient", model)
analysis = analyzer.analyze(input_img)
analysis.shape

In [ ]:
img = np.rollaxis(analysis, 0, 4)[:,:,:,:,0]
img.shape

In [ ]:
meanImg = mean_img(get_nifti_image(img,affine,hdr))
meanImg.shape

In [ ]:
analysis[:,:,:,:,0].shape
# meanImg = mean_img(analysis)